In [3]:
import requests as rq
from bs4 import BeautifulSoup
URL = 'https://yellow.place/ru/r/usa/new-york-ny/shopping/musical-instrument-store'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.86 YaBrowser/21.3.1.186 Yowser/2.5 Safari/537.36',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}


def get_html(url):
    response = rq.get(URL,headers = HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

ar = get_html(URL)
#print(ar)

In [4]:
from time import sleep
def get_content(a):
    items = a.find_all('div', class_ = 'card card-place')
    shops = []
    for item in items:
        shops.append(
        {
            'name' : item.find('h5').get_text().strip(),
            'adress' : item.find('p').get_text().strip()
        })
    
    return shops
    
shops_list = get_content(ar)


In [5]:
def nom_geo(address, format = 'json',limit=1, **kwargs):
   
    params = {'q' : address, 'format' : format, 'limit' : limit, **kwargs}
    response = rq.get(url='https://nominatim.openstreetmap.org/search?',params = params)
    response.raise_for_status()
    sleep(1)
    return response.json()


In [6]:
shop_points = [[] for _ in range(len(shops_list))]
for i in range(len(shops_list)):
    ppoint = nom_geo(shops_list[i]['adress'])
    if ppoint == []:
        continue
    shop_points[i] = [shops_list[i]['name'],ppoint[0]['lat'],ppoint[0]['lon']]

In [7]:
import folium

In [8]:
map_shops = folium.Map(location = [40.5,-73.7],zoom_start = 9)

for shop in shop_points:
    if shop != []:
        folium.Marker(location = [shop[1],shop[2]],popup=shop[0],tooltip='click').add_to(map_shops)

map_shops